In [1]:

from openslide import OpenSlide
from PIL import Image
import numpy as np

import os
from time import time
from joblib import Parallel, delayed
import cv2

In [2]:
svs_src = r'\\babyserverdw4\Digital pathology image lib\obesity study Cristian and Jody set2\raw scanned image'
svss = [os.path.join(svs_src,_) for _ in os.listdir(svs_src) if _.endswith('ndpi')]
svs_dst = r'\\babyserverdw4\Digital pathology image lib\obesity study Cristian and Jody set2\raw scanned image\5x'

In [3]:
def svs2tiff(svs):
    src,fn = os.path.split(svs)
    fn,ext = os.path.splitext(fn)
    fn1 = fn + '.tif'
    fn2 = fn + '.jp2'
    print(fn)
    if os.path.exists(os.path.join(svs_dst,fn)): return
    svs_obj = OpenSlide(svs)
    svs_img = svs_obj.read_region(location=(0,0),level=0,size=svs_obj.level_dimensions[0]).convert('RGB')
    resize_factorx = 2/float(svs_obj.properties['openslide.mpp-x']) #2um=5x, 1um=10x, 0.5um=20x, 0.25um=40x
    resize_factory = 2/float(svs_obj.properties['openslide.mpp-y'])
    resize_dimension = tuple([int(np.ceil(svs_obj.dimensions[0]/resize_factorx)),int(np.ceil(svs_obj.dimensions[1]/resize_factory))])
    svs_img = svs_img.resize(resize_dimension,resample=Image.NEAREST)
    # svs_img.save(os.path.join(svs_dst,fn1),resolution=1,resolution_unit=1,quality=100,compression=None)
    svs_img2 = np.array(svs_img)
    retval = cv2.imwrite(os.path.join(svs_dst,fn2),svs_img2,[cv2.IMWRITE_JPEG2000_COMPRESSION_X1000,800])


In [ ]:
svss2=svss[0:10]

wf24_b5_01.tif
wf26_b5_01.tif
wf27_b5_01.tif
wf45_b5_01.tif
wf46_b5_01.tif
wf53_b5_01.tif
wf56_b5_01.tif
wf59_b5_01.tif
wf61_b5_01.tif
wf61_b5_02.tif
wf62_b5_01.tif


In [ ]:
st = time()
# Parallel(n_jobs=5, prefer="threads")(delayed(svs2tiff)(svs) for svs in svss)
[svs2tiff(svs) for svs in svss2]

In [ ]:
print("{:.2f} sec elapsed for {:d} images at 10x".format(time()-st,len(svss)))